<a href="https://colab.research.google.com/github/zhousanfu/machine-learning-demo/blob/master/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence_transformers sentencepiece cpm_kernels

In [ ]:
!pip install langchain faiss-cpu

In [ ]:
!pip install google-search-results -i pypi.douban.com/simple --trusted-host pypi.douban.com

## Chatglm

In [ ]:
from transformers import AutoTokenizer, AutoModel
from typing import Any, List, Mapping, Optional

class chatGLM():
    def __init__(self, model_name, quantization_bit=4) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True).half().cuda().eval()
        self.model = self.model.quantize(quantization_bit)

    def __call__(self, prompt, history) -> Any:
        response, history = self.model.chat(self.tokenizer , prompt, history=history) # 这里演示未使用流式接口. stream_chat()
        return response, history

llm = chatGLM(model_name="THUDM/chatglm-6B-int4")

In [ ]:
response, history = llm(prompt="你好", history=[])
print("response: %s"%response)
response, history = llm(prompt="我最近有点失眠怎么办?", history=history)
print("response: %s"%response)

## Langchain

LangChain是一个强大的框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源

In [ ]:
# # 官方llm使用OPENAI 接口
# from langchain.llms import OpenAI
# llm = OpenAI(model_name="text-davinci-003")
# prompt = "你好"
# response = llm(prompt)

### prompt
填入内容来引导大模型输出

In [ ]:
from langchain import PromptTemplate



template = """什么是{query},还有如何真正做到并细说实现步骤"""
prompt_tem = PromptTemplate(input_variables=["query"], template=template)
prompt = prompt_tem.format(query='阶级跳跃')

prompt

### Chains
链接多个组件处理一个特定的下游任务

In [ ]:
# from langchain.chains import LLMChain
# chain = LLMChain(llm=openAI(), prompt=promptTem)
# print(chain.run("你好"))

from langchain.chains.base import Chain



class DemoChain():
    def __init__(self, llm, prompt, history) -> None:
        self.llm = llm
        self.prompt = prompt
        self.history = history

    def run(self, query, history, context=None) -> Any:
        if context is not None:
            prompt = self.prompt.format(query=query, context=context)
        else:
            prompt = self.prompt.format(query=query)

        response, history = self.llm(prompt, history)
        return response, history

chain = DemoChain(llm=llm, prompt=prompt_tem, history=[])
response, history = chain.run(query="阶级跳跃", history=[])
print(response, history)

### Embedding
外部信息编码成一个高维向量

In [ ]:
# #官方示例代码，用的OpenAI的ada的文本Embedding模型
# #1） Embeding model
# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model_name="ada")
# query_result = embeddings.embed_query("你好")

# #2) 文本切割
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100, chunk_overlap=0
# )
# texts = """阶级跳跃是指一个人或一个组织通过提高自己的技能、知识和领导能力，从一个阶级跨越到另一个阶级的过程。要实现阶级跳跃，以下是一些建议：\n\n1. 学习新技能：学习新技能可以让人具备新的知识和技能，从而增加自己的竞争力。可以选择学习与目前工作相关的新技能，或者学习与未来工作相关的新技能。\n\n2. 提高知识水平：不断学习新知识可以增加自己的知识储备，从而提高自己的竞争力。可以通过阅读书籍、参加培训、参与线上课程等方式来提高自己的知识水平。\n\n3. 建立良好的人际关系：建立良好的人际关系可以让人更容易得到新机会，同时也可以获得更多的支持和帮助。可以通过参加社交活动、建立人脉、参加社区组织等方式来建立良好的人际关系。\n\n4. 提高自己的领导能力：领导能力可以让人更好地管理自己的时间和资源，从而更好地完成工作。可以通过参加领导力课程、参加团队建设活动、自我反思等方式来提高自己的领导能力。\n\n5. 建立自己的品牌：建立自己的品牌可以让人更容易被人记住，从而更容易得到新机会。可以通过写博客、发布视频、制作网站等方式来建立自己的品牌。\n\n阶级跳跃需要长期的努力和不断学习，需要对自己的能力和目标有清晰的认识，并制定明确的计划和目标。"""
# texts = text_splitter.create_documents([texts])
# print(texts[0].page_content)

# # 3)入库检索，官方使用的Pinecone,他提供一个后台管理界面 | 用户需求太大，不好用了已经，一直加载中....
# import pinecone
# from langchain.vectorstores import Pinecone
# pinecone.init(api_key=os.getenv(""), enviroment=os.getenv(""))

# index_name = "demo"
# search = Pinecone.from_documents(texts=texts, embeddings, index_name=index_name)
# query = "What is magical about an autoencoder?"
# result = search.similarity_search(query)

### 这里使用chatGLM
# 1） Embedding model:  text2vec-large-chinese

In [ ]:
import numpy as np
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS


class TextSpliter(CharacterTextSplitter):
    def __init__(self, separator: str = "\n\n", **kwargs: Any):
        super().__init__(separator, **kwargs)

    def split_text(self, text: str) -> List[str]:
        texts = text.split("\n")
        texts = [Document(page_content=text, metadata={"from": "filename or book.txt"}) for text in texts]
        return texts

texts = response

text_splitter = TextSpliter()
texts = text_splitter.split_text(texts)
texts1 = [text.page_content for text in texts]

texts1

In [ ]:
model_name = "GanymedeNil/text2vec-large-chinese"
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={'device': "cuda"})
query_result = embeddings.embed_query("阶级跳跃")

np.array(query_result).shape

In [ ]:
vs_path = "text_to_emb"

docs = embeddings.embed_documents(texts1)

vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local(vs_path)

vector_store = FAISS.load_local(vs_path, embeddings)
related_docs_with_score = vector_store.similarity_search_with_score(query="阶级跳跃", k=2)

related_docs_with_score

In [ ]:
# 基于查询到的知识做prompt
context = ""
for pack in related_docs_with_score:
    doc, socre = pack
    content = doc.page_content
    print("检索到的知识=%s, from=%s, socre=%.3f"%(content, doc.metadata.get("from"), socre))
    context += content

In [ ]:
# 重新配置一个基于上下文的模板在来调下语言模型
template = "已知{context}, 请给我解释一下{query}的意思?"
promptTem = PromptTemplate(input_variables=["context", "query"], template=template)
chain = DemoChain(llm=llm, prompt=promptTem, history=[])
print("-"*80)
print(chain.run(query="阶级跳跃", context=context, history=[]))
print("-"*80)

### llm重写

TfboyLLM继承了langchain.llms.base的LLM类。需要实现它的两个方法：

*   _call: 主要的处理方法，对传来的prompt问题分析，给他一个答案。return

*   _identifying_params: 说明LLM类中的参数和数值。本例中没有类的成员变量。


其实关键要看_call中实现的逻辑：
收到prompt先打印出来。
对问题正则匹配，规则为：[数字]+[运算符]+[数字]。匹配到，返回计算结果。匹配不到继续执行。
判断有没有[?]。如果有，则对文本中字符进行替换，规则为：我->你, 你->我, 吗->"", ?->!。
如果都不符合，就返回：“很抱歉，请换一种问法。比如：1+1等于几”。

In [ ]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
import re

class TfboyLLM(LLM):

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        print("问题:",prompt)
        pattern = re.compile(r'^.*(\d+[*/+-]\d+).*$')
        match = pattern.search(prompt)
        if match:
            result = eval(match.group(1))
        elif "？" in prompt:
            rep_args = {"我":"你", "你":"我", "吗":"", "？":"！"}
            result = [(rep_args[c] if c in rep_args else c) for c in list(prompt)]
            result = ''.join(result)
        else:
            result = "很抱歉，请换一种问法。比如：1+1等于几"
        return result

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}



In [ ]:
llm = TfboyLLM()
print("答案:",llm("我能问你问题吗？"))